## Let's answer some questions and visualise some data

### By using pandas and plotly

In [2]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connecting to the WizardsCo database
connection = sqlite3.connect('wizards_co.db')
cursor = connection.cursor()

### What is the number of installations that the company is doing every month?

In [3]:
monthly_installations_query = """SELECT COUNT(*) AS number_of_installations, installation_month
                FROM fact_installation
                GROUP BY installation_month
                ORDER BY installation_month;"""

monthly_installations = pd.read_sql_query(monthly_installations_query, connection)

monthly_installations

,number_of_installations,installation_month
0,2,04
1,2,05
2,1,06
3,5,07
4,8,08
5,8,09
6,10,10


In [49]:
fig1 = px.bar(monthly_installations, x='installation_month', y='number_of_installations', text_auto=True, title='Total number of installations per month')
fig1.write_image("images/installations_per_month.jpeg")
fig1.show()

![](images/installations_per_month.jpeg)

### Which product category brings us more revenues?

In [5]:
revenues_per_category_query = """SELECT SUM(price) AS revenues, product_category
                            FROM fact_installation
                            GROUP BY product_category
                            ORDER BY revenues DESC"""

revenues_per_category = pd.read_sql_query(revenues_per_category_query, connection)

revenues_per_category

,revenues,product_category
0,111105.0,Dangerous Items
1,7101.0,IT & Network Stuff
2,570.0,Beauty Accessories
3,36.0,Miscellaneous
4,12.0,Medical Device


In [50]:
fig2 = px.pie(revenues_per_category, values='revenues', names='product_category', title='Revenues per product category')
fig2.write_image("images/revenues_per_category.jpeg")
fig2.show()

![](images/revenues_per_category.jpeg)

### Which region of the world is our best market?

In [7]:
revenues_per_region_query = """SELECT SUM(price) AS revenues, region
                            FROM fact_installation
                            GROUP BY region
                            ORDER BY revenues DESC"""

revenues_per_region = pd.read_sql_query(revenues_per_region_query, connection)

revenues_per_region

,revenues,region
0,44703.0,Europe
1,37071.0,America
2,37050.0,Asia


In [51]:
fig3 = px.bar(revenues_per_region, x='region', y='revenues', text_auto=True, title='Total revenues per region')
fig3.write_image("images/revenues_per_region.jpeg")
fig3.show()

![](images/revenues_per_region.jpeg)

In [53]:
#let's visualise that last one differently, using a plotly map

#since plotly doesn't allow a representation by continent, I've chosen one big country per region
iso_region = {'Europe':'FRA','America':'USA','Asia':'CHN'}
revenues_per_region['iso_region'] = revenues_per_region['region'].map(iso_region)

revenues_per_region

fig_map = px.choropleth(revenues_per_region, 
                     locations=revenues_per_region["iso_region"],
                     locationmode= 'ISO-3',
                     color="revenues",
                     projection="natural earth",
                     title='Total revenues per region (map)')
fig_map.write_image('images/revenues_per_region_map.jpeg')
fig_map.show()

![](images/revenues_per_region_map.jpeg)

### How is the revenue from premium customers compared to regular ones?

In [10]:
premium_users_query = """SELECT SUM(price) AS total_revenues, AVG(price) AS average_revenues, premium_customer
                            FROM fact_installation
                            GROUP BY premium_customer
                            ORDER BY total_revenues DESC"""

premium_users = pd.read_sql_query(premium_users_query, connection)

premium_users

,total_revenues,average_revenues,premium_customer
0,77004.0,3666.857143,yes
1,41820.0,2788.000000,no


In [54]:
fig4 = px.bar(premium_users, x='premium_customer', y='total_revenues', color='premium_customer', text_auto=True, title='Total revenues per type of customer')
fig4.write_image('images/premium_total.jpeg')
fig4.show()

In [55]:
fig4b = px.bar(premium_users, x='premium_customer', y='average_revenues', color='premium_customer', text_auto=True, title='Average revenues per type of customer')
fig4b.write_image('images/premium_average.jpeg')
fig4b.show()

![](images/premium_total.jpeg) ![](images/premium_average.jpeg)

### How's the growth per region in terms of revenue?

In [26]:
region_growth_query = """SELECT SUM(price) AS revenue, region, installation_month
                        FROM fact_installation
                        GROUP BY region, installation_month
                        ORDER BY installation_month;"""

region_growth = pd.read_sql_query(region_growth_query, connection)

region_growth

,revenue,region,installation_month
0,95.0,Europe,04
1,12345.0,America,05
2,5.0,Europe,05
3,5.0,Europe,06
4,12.0,America,07
5,12350.0,Asia,07
6,879.0,Europe,07
7,24.0,America,08
8,5.0,Asia,08
9,15501.0,Europe,08


In [56]:
fig5 = px.bar(region_growth, x='installation_month', y='revenue', color='region', barmode='group', text_auto=True, title='Revenues per month per region')
fig5.write_image('images/region_growth.jpeg')
fig5.show()

![](images/region_growth.jpeg)

### And in terms of number of sales?

In [33]:
region_growth_query2 = """SELECT COUNT(*) AS sales, region, installation_month
                        FROM fact_installation
                        GROUP BY region, installation_month
                        ORDER BY installation_month;"""

region_growth2 = pd.read_sql_query(region_growth_query2, connection)

region_growth2

,sales,region,installation_month
0,2,Europe,04
1,1,America,05
2,1,Europe,05
3,1,Europe,06
4,1,America,07
5,2,Asia,07
6,2,Europe,07
7,2,America,08
8,1,Asia,08
9,5,Europe,08


In [57]:
fig5 = px.bar(region_growth2, x='installation_month', y='sales', color='region', barmode='group', text_auto=True, title='Number of installations per month per region')
fig5.write_image('images/region_growth2.jpeg')
fig5.show()

![](images/region_growth2.jpeg)